In [1]:
import torch
import os
import numpy as np
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import KMeansSMOTE

from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
from torch.utils.data import Dataset,DataLoader
import gc
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [2]:
import os
def readDir(dirPath):
    if dirPath[-1] == '/':
        print(u'errorpath/')
        return
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        for f in fileList:
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readDir(f)
                allFiles = subFiles + allFiles 
            else:
                allFiles.append(f)
        return allFiles
    else:
        return 'Error,not a dir'
    
def props_to_onehot(props):
    props=props.detach().cpu().numpy()
    a=np.argmax(props,axis=1)
    b=np.zeros((len(a),props.shape[1]))
    b[np.arange(len(a)),a]=1
    return b

In [4]:
test_percent=0.25
batch_size=200
learnrate=0.001
countEpoch=500

allfiles=readDir('Enc\\Enc.02')
allfiles.sort()
res_Prediction=[]
for file in allfiles:
    course_name=file[11:20]
    code_size=file[21:23]
#     percent=file[25:28]
    res_thisCourse=[]
    res_thisCourse.append(course_name)
#     res_thisCourse.append(percent)
    res_thisCourse.append(code_size)
    print(course_name +'-'+code_size )
    df=pd.read_csv(file)
    x=df.iloc[:,0:-1]
    x=x.values
    
    targets=df.iloc[:,-1]
    ls_x,ls_test_x,ls_y,ls_testy = train_test_split(x,targets,test_size=test_percent)
    
    try:
        ls_x,ls_y=KMeansSMOTE().fit_resample(ls_x,ls_y)
    except BaseException:
        continue
    
    train_input = torch.from_numpy(ls_x)
    train_input = train_input.to(torch.float32).to(device)
    train_target= torch.from_numpy(np.array(pd.get_dummies(pd.Series(ls_y))))
    train_target = train_target.to(torch.float32).to(device)
    
    test_input = torch.from_numpy(ls_test_x)
    test_target=torch.from_numpy(np.array(pd.get_dummies(pd.Series(ls_testy))))
    test_input = test_input.to(torch.float32).to(device)
    test_target = test_target.to(torch.float32).to(device)
    
    print(train_input.size(),train_target.size(),test_input.size(),test_target.size())
    
    countALLSamples = train_input.size()[0] 
    countFeatures = train_input.size()[1]
    countTargets=test_target.size()[1]
    
    class MData(Dataset):
        def __init__(self,x,y):
            self.data=x
            self._x=x
            self._y=y
        def __getitem__(self,index):
            return self._x[index],self._y[index]
        def __len__(self):
            return len(self.data)
    mydata=MData(train_input,train_target)
    dataloader=DataLoader(mydata,batch_size=batch_size,shuffle=False,drop_last=False,num_workers=0)

    class FNN(nn.Module):
        def __init__(self, input_dim):
            super(FNN, self).__init__()
            self.net = nn.Sequential(
                nn.Linear(input_dim, input_dim),
                nn.ReLU(),
                nn.Linear(input_dim, 512),
                nn.ReLU(),
                nn.Linear(512, 512),
                nn.ReLU(),
                nn.Linear(512, 256),
                nn.ReLU(),
                nn.Linear(256, 128),
                nn.ReLU(),
                nn.Linear(128, 64),
                nn.ReLU(),
                nn.Linear(64, countTargets),
                nn.Softmax(dim=1)
            )
            self.criterion = nn.CrossEntropyLoss()
        def forward(self, x):
            return self.net(x)
    
    model=FNN(countFeatures)
    model = model.to(device)
    model.criterion = model.criterion.to(device)
    
    gc.collect()
    torch.cuda.empty_cache()
    optimizer = torch.optim.Adam(model.parameters(),lr=learnrate)
    for epoch in range(countEpoch):
        gc.collect()
        torch.cuda.empty_cache()
        for i,data in enumerate(dataloader):
            x,y=data
            yhat=model(x)
            loss=model.criterion(yhat,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            gc.collect()
            torch.cuda.empty_cache()
        with torch.no_grad():
            yhat=model(train_input)
            allloss=model.criterion(yhat,train_target)
            if(epoch%100==0):
                print('Epoch '+str(epoch) + '.loss:'+ str(allloss.item()))
    
    test_y=model(test_input)
    test_y=props_to_onehot(test_y)
    test_target=test_target.cpu().numpy()
   
    res_thisCourse.append(sk.metrics.accuracy_score(test_target,test_y))
    res_thisCourse.append(sk.metrics.precision_score(test_target,test_y,average='macro'))
    res_thisCourse.append(sk.metrics.recall_score(test_target,test_y,average='macro'))
    res_thisCourse.append(sk.metrics.f1_score(test_target,test_y,average='macro'))
    print('Accuracy: ' + str(sk.metrics.accuracy_score(test_target,test_y)))
    print('Precision: ' + str(sk.metrics.precision_score(test_target,test_y,average='macro')))
    print('Recall: ' + str(sk.metrics.recall_score(test_target,test_y,average='macro')))
    print('f1_score: ' + str(sk.metrics.f1_score(test_target,test_y,average='macro')))
    res_Prediction.append(res_thisCourse)
dfRes=pd.DataFrame(res_Prediction)
dfRes.to_csv("OULA.Exp.Res\\Enc.KMeansSMOTE.Prediction.Res.csv",index=False)
print('ALL Completed!')

AAA_2013J-12


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=2
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=2
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048

torch.Size([777, 12]) torch.Size([777, 4]) torch.Size([95, 12]) torch.Size([95, 4])
Epoch 0.loss:1.3818368911743164
Epoch 100.loss:0.9081253409385681
Epoch 200.loss:0.9347513318061829
Epoch 300.loss:0.9491334557533264
Epoch 400.loss:0.8922344446182251
Accuracy: 0.5789473684210527
Precision: 0.3476890756302521
Recall: 0.35121984649122806
f1_score: 0.34091826808362236
AAA_2013J-18


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=2
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=2
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048

torch.Size([749, 18]) torch.Size([749, 4]) torch.Size([95, 18]) torch.Size([95, 4])
Epoch 0.loss:1.380088210105896
Epoch 100.loss:1.0020257234573364
Epoch 200.loss:0.968529224395752
Epoch 300.loss:0.9329368472099304
Epoch 400.loss:0.9195305109024048
Accuracy: 0.6210526315789474
Precision: 0.5189655172413793
Recall: 0.4606807511737089
f1_score: 0.4431047448176475
AAA_2013J-24


...

Precision: 0.316990451881263
Recall: 0.39812335022446577
f1_score: 0.3247357049661477
GGG_2014J-6.


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=3
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=3
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048

torch.Size([980, 6]) torch.Size([980, 4]) torch.Size([174, 6]) torch.Size([174, 4])
Epoch 0.loss:1.3835749626159668
Epoch 100.loss:1.0466786623001099
Epoch 200.loss:1.1086313724517822
Epoch 300.loss:1.0686639547348022
Epoch 400.loss:1.0883923768997192
Accuracy: 0.39080459770114945
Precision: 0.34951037994516254
Recall: 0.35695630473027734
f1_score: 0.3523500309214595
GGG_2014J-9.


C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=3
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048 or by setting the environment variable OMP_NUM_THREADS=3
  "MiniBatchKMeans is known to have a memory leak on "
C:\Users\Xiaoven\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1044: UserWarning: MiniBatchKMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can prevent it by setting batch_size >= 2048

torch.Size([980, 9]) torch.Size([980, 4]) torch.Size([174, 9]) torch.Size([174, 4])
Epoch 0.loss:1.3830366134643555
Epoch 100.loss:1.082930326461792
Epoch 200.loss:1.0311261415481567
Epoch 300.loss:1.0008294582366943
Epoch 400.loss:0.9881218075752258
Accuracy: 0.46551724137931033
Precision: 0.432097966614748
Recall: 0.42629326533436124
f1_score: 0.4252263137706176
ALL Completed!
